In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
!pip install ipywidgets
!pip install opendatasets ipyopenlayers pandas

from ipyopenlayers import RasterTileLayer, ZoomSlider, Map
from ipyleaflet import Map, Marker, MarkerCluster,Popup
from ipywidgets import HTML
import opendatasets as od
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import time
import os
import kagglehub
from tqdm import tqdm
import pickle

!pip install pyspark opendatasets kagglehub

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring, avg, when, lit, min as spark_min, max as spark_max

spark = SparkSession.builder \
    .appName("QOLDataProcessing") \
    .getOrCreate()


import kagglehub
import zipfile

# # !kaggle datasets download -d guslovesmath/us-pollution-data-200-to-2022 -p ./ --unzip
# path = kagglehub.dataset_download("guslovesmath/us-pollution-data-200-to-2022")
# with zipfile.ZipFile("us-pollution-data-200-to-2022.zip", 'r') as zip_ref:
#     zip_ref.extractall("data")
file_path = "QOL.csv"




In [ ]:
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import FloatType
file_path = "QOL.csv"


df = spark.read.option("header", True).csv(file_path)
# Select a quality of life indicator for marker coloring (e.g., Crime Rate)
indicator = 'Unemployment'

# Remove '%' and convert to float
df = df.withColumn(indicator, regexp_replace(col(indicator), '%', '').cast(FloatType()))



In [ ]:
from pyspark.sql.functions import mean
from pyspark.sql import functions as F

state_unemployment_avg = df.groupBy("LSTATE").agg(mean(indicator).alias("avg_unemployment"))
state_unemployment_avg = state_unemployment_avg.withColumnRenamed("LSTATE", "state")

# Salvează în CSV
state_unemployment_avg.write.csv('state_unemployment_avg.csv', header=True, mode='overwrite')

# Selectăm coloanele necesare
rankings_df = df.select("LSTATE", "NMCNTY", indicator, "countyhelper")

# Redenumim coloanele pentru claritate
rankings_df = rankings_df.withColumnRenamed("LSTATE", "state") \
                         .withColumnRenamed("NMCNTY", "county") \
                         .withColumnRenamed(indicator, "Unemployment")

# Calculăm valorile minime și maxime pentru scorul de ranking
min_unemployment = rankings_df.agg(F.min("Unemployment")).collect()[0][0]
max_unemployment = rankings_df.agg(F.max("Unemployment")).collect()[0][0]

# Calculăm scorul de ranking
rankings_df = rankings_df.withColumn("ranking", 1 - ((F.col("Unemployment") - min_unemployment) / (max_unemployment - min_unemployment)))

# Salvăm rezultatul într-un fișier CSV
rankings_df.write.csv("unemployment_rankings.csv", header=True, mode="overwrite")


In [ ]:
# Set up geolocator with increased timeout
geolocator = Nominatim(user_agent="qof_map_app", timeout=10)

# Load cache if it exists
cache_file = "geocode_cache.pkl"
if os.path.exists(cache_file):
    with open(cache_file, "rb") as f:
        geocode_cache = pickle.load(f)
else:
    geocode_cache = {}

# Create a dictionary to store locations not found
not_found_cache = {}

# Define geocoding function with caching
def geocode_location(row):
    row['county'] = row['county'].replace(' County', '')
    row['county'] = row['county'].replace(' city', '')
    row['county'] = row['county'].replace(' Census Area', '')
    row['county'] = row['county'].replace('District of Columbia', 'Washington')
    county_state = f"{row['county']}, {row['state']}"
    if county_state in geocode_cache:
        return geocode_cache[county_state]
    if county_state in not_found_cache:
        return None

    try:
        if row['county'] == 'Not in a county':
            # Geocode using only state
            location = geolocator.geocode({"state": row['state'], "country": "USA"})
            if location:
                point = Point(location.longitude, location.latitude)
                geocode_cache[county_state] = point
                return point
            else:
                print(f"Location not found for state: {row['state']}")
                not_found_cache[county_state] = None
                return None
        else:
            # Geocode using county and state
            location = geolocator.geocode({"county": row['county'], "state": row['state'], "country": "USA"})
            if location:
                point = Point(location.longitude, location.latitude)
                geocode_cache[county_state] = point
                return point
            else:
                print(f"Location not found: {county_state}")
                # Attempt to geocode using only state
                location2 = geolocator.geocode({"state": row['state'], "country": "USA"})
                if location2:
                    point = Point(location2.longitude, location2.latitude)
                    geocode_cache[county_state] = point
                    return point
                else:
                    not_found_cache[county_state] = None
                return None
    except Exception as e:
        print(f"Geocoding error for {county_state}: {e}")
        time.sleep(1)
        return None

# Apply geocoding with a progress bar
aggregated_df_pd = rankings_df.toPandas()
aggregated_df_pd['geometry'] = [geocode_location(row) for _, row in tqdm(aggregated_df_pd.iterrows(), total=len(aggregated_df_pd))]

# Save cache for future use
with open(cache_file, "wb") as f:
    pickle.dump(geocode_cache, f)

# Save not found cache for future reference
not_found_cache_file = "not_found_cache.pkl"
with open(not_found_cache_file, "wb") as f:
    pickle.dump(not_found_cache, f)

# Drop rows with missing geometry
df = aggregated_df_pd.dropna(subset=['geometry'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

 19%|█▊        | 583/3134 [09:32<42:11,  1.01it/s]

Location not found: Taylor, GA


 25%|██▍       | 777/3134 [12:35<39:59,  1.02s/it]

Location not found: Jones, GA


 30%|███       | 943/3134 [15:18<36:12,  1.01it/s]

...

 99%|█████████▉| 3107/3134 [50:43<00:26,  1.00it/s]

Location not found: Franklin, OH


 99%|█████████▉| 3112/3134 [50:49<00:23,  1.06s/it]

Location not found: Kings, NY


100%|██████████| 3134/3134 [51:13<00:00,  1.02it/s]


In [ ]:
gdf.to_csv("qof_data_spark_geocode.csv", index=False)


In [ ]:
print("Valoare minimă Unemployment:", gdf["Unemployment"].min())
print("Valoare maximă Unemployment:", gdf["Unemployment"].max())

Valoare minimă Unemployment: -1.0
Valoare maximă Unemployment: 17.190000534057617
